<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

In [2]:
try:
    data = pd.read_csv("insurance.csv")
except:
    print("could not open the file")

In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
data['Возраст'].unique()

array([41., 46., 29., 21., 28., 43., 39., 25., 36., 32., 38., 23., 40.,
       34., 26., 42., 27., 33., 47., 30., 19., 31., 22., 20., 24., 18.,
       37., 48., 45., 44., 52., 49., 35., 56., 65., 55., 57., 54., 50.,
       53., 51., 58., 59., 60., 61., 62.])

In [6]:
data['Пол'].unique()

array([1, 0])

In [7]:
data['Члены семьи'].unique()

array([1, 0, 2, 4, 3, 5, 6])

In [8]:
data['Страховые выплаты'].unique()

array([0, 1, 2, 3, 5, 4])

Все данные числовые, пропусков нет. Для уменьшения нагрузки на ресурс, я бы заменила тип данных в столбце "возраст". Поскольку все значения в столбце целочисленные, то float здесь избыточен. Заменим его на int_8. Судя по тому, какие уникальные значения хранятся в столбцах "Пол", "Члены семьи" и "Страховые выплаты", int_8 будет вполне. 

In [9]:
try:
    data['Возраст'].astype('UInt8')
except:
    print("Не удалось изменить тип данных в столбце Возраст")

try:
    data['Пол'].astype('UInt8')
except:
    print("Не удалось изменить тип данных в столбце Пол")
    
try:
    data['Члены семьи'].astype('UInt8')
except:
    print("Не удалось изменить тип данных в столбце Члены семьи")

try:
    data['Страховые выплаты'].astype('UInt8')
except:
    print("Не удалось изменить тип данных в столбце Страховые выплаты")


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


memory usage уменьшилось с  195 до 78 КВ.

После изучения архива, были изменены типы данных с целью уменьшения нагрузки на ресурс.

## Умножение матриц

Разобъем выборку на target и features

In [11]:
target = data['Страховые выплаты']
features = data.drop('Страховые выплаты',axis = 1)

Теперь на тестовую и обучающую:

In [12]:
rs = 12345

In [13]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.25, random_state  = rs)

Преобразуем features в матрицу

In [14]:
vector_test = features_test.values
vector_train = features_train.values

Обозначения:

 - *𝑋*
  — матрица признаков (нулевой столбец состоит из единиц)

 - *𝑦*
  — вектор целевого признака

 - *𝑃*
  — матрица, на которую умножаются признаки

 - *𝑤*
  — вектор весов линейной регрессии (нулевой элемент равен сдвигу)
 - *E* - единичная матрица   
  **Вопрос:**
  Изменится ли качество линейной регрессии, если признаки умножить на обратимую матрицу.

  **Ответ:**
  не изменится.

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$   

**Обоснование:**


Пускай P - обратимая квадратная матрица шириной Х.


   Поскольку:
   $$
   a = Xw
   $$
   Следовательно:
   $$
   a = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w_1
   $$
   Формула обучения:
   $$
   w = (X^TX)^{-1}X^Ty
   $$
   Значит:
   $$
   w_1 = ((XP)^TXP)^{-1}(XP)^Ty
   $$
   
   $$
   w_1 = (P^TX^TXP)^{-1}(XP)^Ty
   $$
   $$
   w_1 = P^{-1}(P^TX^TX)^{-1}P^TX^Ty
   $$
   $$
   w_1 = P^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty
   $$
   $$
   w_1 = P^{-1}(X^TX)^{-1}X^Ty
   $$   
   
   $$
   w_1 = \arg\min ||X_1w_1-y||^2
   $$
   $$
   w_1 = \arg\min ||XPP^{-1}(X^TX)^{-1}X^Ty - y||^2
   $$
   
   $$
   w_1 = \arg\min ||X(X^TX)^{-1}X^Ty - y||^2
   $$
   поскольку
   $$
   w = (X^TX)^{-1}X^Ty 
   $$
   то
   $$
   w_1 = \arg\min ||Xw - y||^2
   $$
   $$w_1 = w$$
            

## Алгоритм преобразования

**Алгоритм** Алгоритм следующий: получаем матрицу, проверяем её на обратимость, преобразуем матрицу, проверяем на преобразованных признаках, используя r2_score

Генерируем случайную матрицу, размерностью 4Х4

In [15]:
random_mrtx = np.random.rand(4,4)
random_mrtx

array([[0.55341244, 0.06217579, 0.36327882, 0.19599436],
       [0.98401569, 0.60815001, 0.12434034, 0.61139434],
       [0.22893955, 0.37676377, 0.88089485, 0.27215232],
       [0.41389363, 0.24195986, 0.8090885 , 0.52978066]])

Проверяем её на обратимость

In [16]:
inv_mrtx = np.linalg.inv(random_mrtx)
inv_mrtx

array([[ 2.26634054,  0.18305407,  0.00692458, -1.05325233],
       [-1.57778323,  1.17444046,  2.25999395, -1.93263422],
       [ 0.77666988, -0.64371529,  0.92273724, -0.01846822],
       [-2.23612991,  0.30369225, -2.44680499,  3.6213039 ]])

Напишем класс Encrypted

In [17]:
class Encrypted:
    def fit(self, features_tr, target_tr):
        self.inv = np.linalg.inv(np.random.rand(4,4))
        self.mult = features_tr @ self.inv
        X = np.concatenate((np.ones((self.mult.shape[0], 1)), self.mult), axis=1)
        y = target_tr
        w = ((np.linalg.inv(X.T.dot(X))) @ X.T) @ y
        self.w = w[1:]
        self.w0 = w[0]
    def predict(self, features_tested):
        result = (features_tested @ self.inv) @ self.w + self.w0
        return result

In [18]:
model = Encrypted()
model.fit(features_train, target_train)
predictions = model.predict(features_test)
result = r2_score(target_test, predictions)
print(result)

0.4352276237497038


**Обоснование**
Поскольку мы только что доказали, что качество линейной регрессии после умножения на обратную матрицу не меняется, то предлагаю использовать это свойство для защиты данных клиента.

## Проверка алгоритма

Для того чтобы проверить алгоритм, сравним качества линейной регрессии из sklearn до и после преобразования. Используем для этого метрику r2.

In [19]:
ordinary_model = LinearRegression()
ordinary_model.fit(features_train, target_train)
ordinary_predict = ordinary_model.predict(features_test)
ordinary_r2 = r2_score(target_test, ordinary_predict)
print("r2 для модели sklearn", ordinary_r2)

encrypted_model = Encrypted()
encrypted_model.fit(features_train, target_train)
encrypted_predict = encrypted_model.predict(features_test)
encrypted_r2 = r2_score(target_test, encrypted_predict)
print("r2 для зашифрованной модели", encrypted_r2)

r2 для модели sklearn 0.43522757127026546
r2 для зашифрованной модели 0.4352275741332471


Как видим до седьмого знака после запятой значения r2 совпадают. Небольшие различия можно списать на погрешность вычислений/округлений, это допустимо. В остальном, можно смело заявлять, что после умножения признаков на обратимую матрицу качество линейной регрессии не меняется, а значит, можно использовать такое умножение для шифрования информации.


## Вывод

После загрузки и изучения часть у части данных были преобразованы типы для меньшей нагрузки на ресурс и большего быстродействия. Также были удалены явные дубликаты.   
Теоретически было доказано, что после умножения признаков на обратимую матрицу предсказание не меняется, т.е качество линейной регрессии остается неизменным.
Был создан и проверен метрикой r2 алгоритм преобразования данных. Была проведена проверка посредством сравнения метрик r2 нашей модели и модели sklearn. Проверка показала одинаковые результаты, что говорит о том, что для шифрования информации о клиентах страховой компании, мы можем использовать умножение